In [1]:
import tkinter as tk
import warnings
from tkinter import filedialog, messagebox
import numpy as np
import psutil
import os
import pickle
from PIL import Image
import sys
import time
import subprocess
import cv2
import leap
import joblib
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error
warnings.filterwarnings('ignore')



In [2]:
_TRACKING_MODES = {
    leap.TrackingMode.Desktop: "Desktop",
    leap.TrackingMode.HMD: "HMD",
    leap.TrackingMode.ScreenTop: "ScreenTop",
}

class GestureCapture:
    def __init__(self, required_frames=30, save_dir="auth_gesture"):
        self.required_frames = required_frames
        self.frames = []
        self.capturing = False
        self.save_dir = save_dir
        self.tracking_mode = leap.TrackingMode.Desktop
        
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

    def start_capture(self):
        """Start capturing a single gesture."""
        self.capturing = True
        self.frames = []  # Reset frames for new gesture
        print("Gesture capture started!")

    def stop_capture(self):
        """Stop capturing and save the gesture."""
        self.capturing = False
        self.save_gesture()
        print("Gesture capture stopped!")

    def save_gesture(self):
        """Save the captured gesture as 'gesture_auth.npy' for authentication."""
        filename = os.path.join(self.save_dir, "gesture_auth.npy")
        np.save(filename, np.array(self.frames))
        print(f"Authentication gesture saved with {len(self.frames)} frames.")

    def process_frame(self, event):
        """Extract joint positions (X, Y, Z) from Leap Motion event."""
        if len(event.hands) == 0:
            return None

        hand_data = []
        for hand in event.hands:
            for finger in hand.digits:
                for bone in finger.bones:
                    joint_position = [bone.prev_joint.x, bone.prev_joint.y, bone.prev_joint.z]
                    hand_data.append(joint_position)
        
        return np.array(hand_data).flatten()  # Flatten the data

    def render_visualization(self, event, canvas):
        """Render hand tracking visualization."""
        canvas.output_image[:, :] = 0  # Clear previous frame

        if len(event.hands) == 0:
            cv2.putText(canvas.output_image, "No hands detected", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
        else:
            for hand in event.hands:
                for finger in hand.digits:
                    for bone in finger.bones:
                        start = canvas.get_joint_position(bone.prev_joint)
                        end = canvas.get_joint_position(bone.next_joint)
                        if start and end:
                            cv2.line(canvas.output_image, start, end, (0, 255, 0), 2)
                            cv2.circle(canvas.output_image, start, 3, (255, 0, 0), -1)
                            cv2.circle(canvas.output_image, end, 3, (255, 0, 0), -1)

        cv2.putText(canvas.output_image, f"Frames: {len(self.frames)}", (50, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        # cv2.imshow("Gesture Capture", canvas.output_image)


class Canvas:
    """Handles the visualization of the hand tracking."""
    def __init__(self):
        self.screen_size = [500, 700]
        self.output_image = np.zeros((self.screen_size[0], self.screen_size[1], 3), np.uint8)

    def get_joint_position(self, bone):
        """Convert Leap Motion bone coordinates to 2D screen positions."""
        if bone:
            return int(bone.x + (self.screen_size[1] / 2)), int(bone.z + (self.screen_size[0] / 2))
        return None

    def set_tracking_mode(self, tracking_mode):
        self.tracking_mode = tracking_mode

    # def toggle_hands_format(self):
    #     self.hands_format = "Dots" if self.hands_format == "Skeleton" else "Skeleton"
    #     print(f"Set hands format to {self.hands_format}")

    # def save_image(self):
    #     filename = f"test_image.png"
    #     cv2.imwrite(filename, self.output_image)
    #     print(f"Image saved as {filename}")

class TrackingListener(leap.Listener):
    def __init__(self, capture, canvas):
        self.canvas = canvas
        self.capture = capture

    # def on_connection_event(self, event):
    #     pass

    # def on_tracking_mode_event(self, event):
    #     self.canvas.set_tracking_mode(event.current_tracking_mode)
    #     print(f"Tracking mode changed to {_TRACKING_MODES[event.current_tracking_mode]}")

    # def on_device_event(self, event):
    #     try:
    #         with event.device.open():
    #             info = event.device.get_info()
    #     except leap.LeapCannotOpenDeviceError:
    #         info = event.device.get_info()

    #     print(f"Found device {info.serial}")

    def on_tracking_event(self, event):
        """Process Leap Motion tracking event."""
        self.capture.render_visualization(event, self.canvas)

        if self.capture.capturing:
            frame_data = self.capture.process_frame(event)
            if frame_data is not None:
                self.capture.frames.append(frame_data)

            if len(self.capture.frames) >= self.capture.required_frames:
                self.capture.stop_capture()  # Stop and save gesture automatically



def main():
    canvas = Canvas()
    print("  x: Exit")
    print("  s: Capture and save image")

    gesture_capture = GestureCapture(required_frames=100)
    tracking_listener = TrackingListener(gesture_capture, canvas)
    connection = leap.Connection()
    connection.add_listener(tracking_listener)

    # running = True

    with connection.open():
        print("Press 's' to start capturing the authentication gesture.")
        connection.set_tracking_mode(leap.TrackingMode.Desktop)
        canvas.set_tracking_mode(leap.TrackingMode.Desktop)

        while True:
            cv2.imshow("Gesture capture", canvas.output_image)
            key = cv2.waitKey(1)

            if key == ord('s'):
                gesture_capture.start_capture()
            elif not gesture_capture.capturing and len(gesture_capture.frames) > 0:
                print("Gesture captured successfully. Exiting...")
                break  # Exit once the gesture is saved

    cv2.destroyAllWindows()


class AppUnlock:
    def __init__(self):
        
        # self.model_path = 'autoencoder_model.pkl'
        self.model_path_x = 'models/autoencoder_X.h5'
        self.model_path_y = 'models/autoencoder_Y.h5'
        self.model_path_z = 'models/autoencoder_Z.h5'

        if (not os.path.exists(self.model_path_x)) or (not os.path.exists(self.model_path_y)) or (not os.path.exists(self.model_path_z)) :
            print("No password image registered! Please run register_password.py first")
            sys.exit(1)

        # self.autoencoder = None
        self.autoencoder_X = None
        self.autoencoder_Y = None
        self.autoencoder_Z = None

        self.threshold = None
        self.todo_app_path = 'todo_app.py'  # Path to your Todo app

        self.root = tk.Tk()
        
        self.root.withdraw()
        # from here
        self.root.wm_attributes('-topmost', True)
        # to here
        self.monitor_todo_app()


    def authenticate(self, autoencoder, scaler, test_data):
        test_data = scaler.transform(test_data.reshape(1, -1))
        reconstructed = autoencoder.predict(test_data)
        error = np.mean(np.abs(test_data - reconstructed))
        return error
    

    def verify_password(self, gesture_file = "auth_gesture/gesture_auth.npy"):
        """Verify password image"""
        try:
            self.autoencoder_X = load_model("models/autoencoder_X.h5", custom_objects={"mse": tf.keras.losses.MeanSquaredError()})
            self.autoencoder_Y = load_model("models/autoencoder_Y.h5", custom_objects={"mse": tf.keras.losses.MeanSquaredError()})
            self.autoencoder_Z = load_model("models/autoencoder_Z.h5", custom_objects={"mse": tf.keras.losses.MeanSquaredError()})

            # Load saved scalers
            scaler_x = joblib.load("models/scaler_x.pkl")
            scaler_y = joblib.load("models/scaler_y.pkl")
            scaler_z = joblib.load("models/scaler_z.pkl")
            
            print('Autoencoders and scalers loaded')

            # if self.autoencoder is None:
            #     with open(self.model_path, 'rb') as f:
            #         self.autoencoder = pickle.load(f)
            #     print('Autoencoder loaded')
        
            messagebox.showinfo("Authentication", "Please capture the password gesture")
        
            self.root.withdraw()
            
            
            main()
            data = np.load(gesture_file)  # Load captured gesture
            num_frames, num_features = data.shape

            # Reshape and split into channels
            num_joints = num_features // 3
            data = data.reshape(num_frames, num_joints, 3)

            X_test = data[:, :, 0]  # X coordinates
            Y_test = data[:, :, 1]  # Y coordinates
            Z_test = data[:, :, 2]  # Z coordinates

            X_test = X_test.reshape(-1, 1)  # Ensure shape matches what scaler was trained on
            Y_test = Y_test.reshape(-1, 1)
            Z_test = Z_test.reshape(-1, 1)

            X_test = scaler_x.transform(X_test).reshape(100, 20)  # Reshape back after scaling
            Y_test = scaler_y.transform(Y_test).reshape(100, 20)
            Z_test = scaler_z.transform(Z_test).reshape(100, 20)

            print("X_test shape:", X_test.shape)
            print("Y_test shape:", Y_test.shape)
            print("Z_test shape:", Z_test.shape)

            # Flatten test data for autoencoder input

            input_dim = X_test.shape[0] * X_test.shape[1]  # 100 * 20 = 600
            X_test = X_test.reshape(1, input_dim)  # Reshape to (1, 600)
            Y_test = Y_test.reshape(1, input_dim)
            Z_test = Z_test.reshape(1, input_dim)

            # Pass through autoencoders
            X_reconstructed = self.autoencoder_X.predict(X_test)
            Y_reconstructed = self.autoencoder_Y.predict(Y_test)
            Z_reconstructed = self.autoencoder_Z.predict(Z_test)
            

            # Compute reconstruction errors (MSE) for each channel
            error_x = mean_squared_error(X_test.flatten(), X_reconstructed.flatten())
            error_y = mean_squared_error(Y_test.flatten(), Y_reconstructed.flatten())
            error_z = mean_squared_error(Z_test.flatten(), Z_reconstructed.flatten())

            avg_error = (error_x + error_y + error_z) / 3

            print(f"Reconstruction Errors: X={error_x:.5f}, Y={error_y:.5f}, Z={error_z:.5f}")
            print(f"Average Error: {avg_error:.5f}")

            # if test_keypoints is not None:
            #     test_sample = np.array(test_keypoints)
            #     error = self.authenticate(self.autoencoder, scaler, test_sample)
            #     print(error)

            with open('threshold.pkl', 'rb') as t:
                THRESHOLD = pickle.load(t)
            
            print(f'Threshold: {THRESHOLD}')
            if avg_error < THRESHOLD:
                    
                print("✅ Gesture recognized! Application unlocked.")
                return True
            else:
                print("❌ Gesture not recognized. Access denied.")
                return False
            
        except Exception as e:
            print(f"Verification failed: {e}")
            return False


    def monitor_todo_app(self):
        """Monitor Todo app process"""
        todo_app_process = None
        
        while True:
            try:
                # Check for todo app process
                # found_process = None
                for proc in psutil.process_iter(['name', 'cmdline']):
                    if (proc.info['cmdline'] and 
                        self.todo_app_path in ' '.join(proc.info['cmdline']) and 
                        (todo_app_process is None or not todo_app_process.is_running())):
                        found_process = proc
                        break
                
                if found_process:
                    print("\nTodo App launch detected - Authentication required")
                    # Immediately kill the process before showing authentication
                    found_process.kill()
                    time.sleep(0.1)  # Small delay to ensure process is killed
                    
                    if self.verify_password():
                        print("Access granted! Launching Todo App!")
                        # Launch Todo app in a new process
                        todo_app_process = subprocess.Popen(
                            ['python', self.todo_app_path],
                            creationflags=subprocess.CREATE_NEW_CONSOLE
                        )
                        break
                    else:
                        print("Access denied!")
                        todo_app_process = None
                            
            except Exception as e:
                # print(f"Error in monitoring: {e}")
                pass
            
            time.sleep(0.1)  # Reduce CPU usage


In [3]:
def AppUnlock_main():
    app = AppUnlock()

if __name__ == "__main__":
    AppUnlock_main()


Todo App launch detected - Authentication required


Autoencoders and scalers loaded
  x: Exit
  s: Capture and save image
Press 's' to start capturing the authentication gesture.
Gesture capture started!
Gesture captured successfully. Exiting...Authentication gesture saved with 100 frames.
Gesture capture stopped!

X_test shape: (100, 20)
Y_test shape: (100, 20)
Z_test shape: (100, 20)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
Reconstruction Errors: X=0.00133, Y=0.00061, Z=0.00764
Average Error: 0.00319
Threshold: 0.006087277304633347
✅ Gesture recognized! Application unlocked.
Access granted! Launching Todo App!
